# Set up

In [1]:
!git clone https://github.com/NLP-Reichman/assignment_1.git
!mv assignment_1/data data
!rm assignment_1/ -r

fatal: destination path 'assignment_1' already exists and is not an empty directory.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [21]:
import json
# from google.colab import files
import pandas as pd
import glob
import numpy as np
import nltk

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

Note - do NOT lowercase the sentences in whi HW.

In [16]:
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  df = pd.concat(map(pd.read_csv, glob.glob('assignment_1/data/*.csv')))
  print(df.head())
  unique_chars = set()
  for i in range(len(df)):
    unique_chars.update(set(df.iloc[i]['tweet_text']))
  return list(unique_chars)
len(preprocess())

             tweet_id                                         tweet_text
0  845395018743459840  RT @ONHERPERlOD: Boyfriends that take pictures...
1  845395017917173760  He got his surgery done today but he's happy w...
2  845395018760306693  @levi_a1998 @mcluber29 I'm doing so much winni...
3  845395018336649216  RT @Rt_YourFavBands: #BandsTournament2017 Roun...
4  845395018751856642     #Merlin oh no she wanted to enchant him my bad


1803

## Part 2
Implement the function *lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [23]:
def lm(lang: str, n: int) -> dict[str, dict[str, float]]:
  '''
  Return a language model for the given lang and n_gram (n)
  :param lang: the language of the model
  :param n: the n_gram value
  :return: a dictionary where the keys are n_grams and the values are dictionaries
  '''
  lm = {}
  df = pd.read_csv('assignment_1/data/' + lang + '.csv')
  # generat the n_grams using nltk
  ngrams = nltk.ngrams(df['tweet_text'], n)
  print(ngrams)
  # count the n_grams
  ngrams_freq = nltk.FreqDist(ngrams)
  print(ngrams_freq)
  # calculate the probabilities
  for ngram in ngrams_freq:
    prefix = ngram[:-1]
    if prefix not in lm:
      lm[prefix] = {}
    if ngram[-1] not in lm[prefix]:
      lm[prefix][ngram[-1]] = ngrams_freq[ngram] / (1/len(ngrams_freq))
    else:
      lm[prefix][ngram[-1]] = ngrams_freq[ngram] / ngrams_freq[prefix]

  print(lm)

  return lm

ngrams_en_2 = lm('en', 2)
print(ngrams_en_2)

<FreqDist with 8984 samples and 8994 outcomes>
{('RT @Michael5SOS: can March 24th be @Calum5SOS appreciation day?',): {'RT @Michael5SOS: can March 24th be @Calum5SOS appreciation day?': 53904.0, "Listen to If I Ain't Got You by Alicia Keys on #AppleMusic.@GovMattBevin #liz https://t.co/lFAUY7SpDv": 8984.0, 'RT @QuackityHQ: I actually forgot how retarded unironic minion pictures are holy fuck https://t.co/kjHAyOCbOv': 8984.0, '@weshampton Yes!! A must purchase!!': 8984.0, 'RT @CliffBreazzy: Y\'all be like "Drake trash Kendrick better"  Like nigga wtf they both good lol. Don\'t gotta hate someone cause you like a…': 8984.0, 'Rather like most of his businesses.  @Shurretep': 8984.0, "RT @MuslimFTrump: When will the progressives wake up to realize that Islam isn't here to integrate, it is here to Dominate? #PrayForLondon…": 8984.0, 'What idiot drops their phone down the decking! That would be me meaning the decking had to be cut up! 🙈': 8984.0, 'RT @zaynmalik: "Still Got Time" is number 1 

## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

In [5]:
def eval(model: dict, target_lang: str) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :return: the perplexity value
  '''
  return -1

## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [6]:
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  return pd.DataFrame()

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution.

In [7]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  return ""

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [8]:
####################
# PLACE TESTS HERE #

####################

In [9]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l

'ls' is not recognized as an internal or external command,
operable program or batch file.
